In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# ==============================================================================
# PASSO A: INSTALAÇÃO DE TODAS AS BIBLIOTECAS NECESSÁRIAS
# ==============================================================================
!pip install google-generativeai gspread pandas pillow -q
# ==============================================================================
# PASSO B: AUTENTICAÇÃO E CONFIGURAÇÃO
# ==============================================================================
import google.generativeai as genai
from google.colab import userdata, auth
from PIL import Image
import os
import json
import pandas as pd
import pprint
import hashlib
import gspread
from google.auth import default
import sys
from datetime import datetime
import time

# --- Autenticação para Google Drive e Google Sheets ---
print("Autorizando acesso ao Google Drive e Sheets...")
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
print("Autorização concluída.")

# --- Configuração da API do Gemini ---
try:
    api_key = userdata.get('GEMINI_API_KEY')
    if not api_key:
        raise ValueError("A chave 'GEMINI_API_KEY' não foi encontrada ou está vazia nos Secrets do Colab.")
    genai.configure(api_key=api_key)
    print("API Key do Gemini configurada com sucesso!")
except Exception as e:
    print(f"\nERRO CRÍTICO: {e}")
    sys.exit()

# ==============================================================================
# PASSO C: CONFIGURAÇÕES E FUNÇÃO DE HASH
# ==============================================================================
URL_DA_PLANILHA = '
CAMINHO_DA_PASTA_DRIVE = '

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

def generate_file_hash(filepath):
    hash_md5 = hashlib.md5()
    with open(filepath, "rb") as f:
        for chunk in iter(lambda: f.read(4096), b""):
            hash_md5.update(chunk)
    return hash_md5.hexdigest()

# ==============================================================================
# PASSO D: FUNÇÃO DE LIMPEZA E FORMATAÇÃO DE DADOS
# ==============================================================================
def clean_and_format_data(dados_brutos):
    dados_limpos = dados_brutos.copy()
    campos_monetarios = ['valor_total_servico', 'base_calculo', 'valor_iss']
    for campo in campos_monetarios:
        valor_str = dados_limpos.get(campo, '0')
        if isinstance(valor_str, str):
            valor_numerico = valor_str.replace('R$', '').strip().replace('.', '').replace(',', '.')
            try:
                dados_limpos[campo] = float(valor_numerico)
            except (ValueError, TypeError):
                dados_limpos[campo] = 0.0
    aliquota_str = dados_limpos.get('aliquota', '0')
    if isinstance(aliquota_str, str):
        valor_numerico = aliquota_str.replace('%', '').strip().replace(',', '.')
        try:
            dados_limpos['aliquota'] = float(valor_numerico) / 100.0
        except (ValueError, TypeError):
            dados_limpos['aliquota'] = 0.0
    data_str = dados_limpos.get('data_hora_emissao', '')
    if isinstance(data_str, str) and data_str:
        try:
            dt_obj = datetime.strptime(data_str, '%d/%m/%Y %H:%M:%S')
            dados_limpos['data_hora_emissao'] = dt_obj.strftime('%Y-%m-%d %H:%M:%S')
        except ValueError:
            dados_limpos['data_hora_emissao'] = data_str
    return dados_limpos

# ==============================================================================
# PASSO E: EXECUÇÃO PRINCIPAL (LÓGICA ATUALIZADA PARA LER IMAGENS E PDFs)
# ==============================================================================
try:
    print(f"\nConectando à planilha: {URL_DA_PLANILHA}")
    spreadsheet = gc.open_by_url(URL_DA_PLANILHA)
    worksheet = spreadsheet.sheet1

    HEADERS = [
        'hash', 'arquivo', 'data_processamento', 'numero_nota', 'data_hora_emissao',
        'codigo_verificacao', 'prestador_cnpj', 'prestador_razao_social',
        'prestador_logradouro', 'prestador_bairro', 'prestador_cep', 'prestador_cidade', 'prestador_uf',
        'prestador_inscricao_municipal', 'tomador_cpf', 'tomador_razao_social',
        'tomador_logradouro', 'tomador_bairro', 'tomador_cep', 'tomador_cidade', 'tomador_uf',
        'tomador_email', 'discriminacao_servicos', 'servico_codigo', 'servico_descricao',
        'valor_total_servico', 'base_calculo', 'aliquota', 'valor_iss'
    ]

    if worksheet.row_count == 0 or worksheet.get('A1') == '':
         worksheet.append_row(HEADERS)
         print("Cabeçalho adicionado à planilha (linha 1).")

    print(f"\nBuscando arquivos de imagem e PDF em: {CAMINHO_DA_PASTA_DRIVE}...")
    arquivos_na_pasta = os.listdir(CAMINHO_DA_PASTA_DRIVE)

    for filename in arquivos_na_pasta:
        # ATUALIZADO: Agora procura também por arquivos .pdf
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.pdf')):
            filepath = os.path.join(CAMINHO_DA_PASTA_DRIVE, filename)
            current_hash = generate_file_hash(filepath)

            print(f"\nVerificando o arquivo '{filename}' (hash: {current_hash})...")
            existing_hashes = set(worksheet.col_values(1)[1:])

            if current_hash in existing_hashes:
                print(f"-> O hash já existe na planilha. Pulando.")
                continue

            print(f"-> Hash não encontrado. Processando novo arquivo...")

            # --- NOVA LÓGICA PARA TRATAR IMAGEM vs PDF ---
            if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                print("   (Tipo: Imagem)")
                # Para imagens, abrimos com a biblioteca PIL
                content_to_process = Image.open(filepath)
            elif filename.lower().endswith('.pdf'):
                print("   (Tipo: PDF)")
                # Para PDFs, fazemos o upload para a API primeiro
                uploaded_file = genai.upload_file(path=filepath, display_name=filename)
                print(f"   Arquivo PDF '{filename}' enviado para a API.")
                content_to_process = uploaded_file
            # ----------------------------------------------------

            model = genai.GenerativeModel('gemini-1.5-flash')

            # Prompt levemente ajustado para "documento"
            prompt = """
            Você é um sistema especialista em extrair informações de Notas Fiscais de Serviço brasileiras.
            Analise o documento da nota fiscal fornecida (pode ser uma imagem ou um PDF) e extraia os campos no formato JSON aninhado abaixo.
            Separe os endereços em logradouro, bairro, cep, cidade e uf.
            {
              "numero_nota": "string", "data_hora_emissao": "string (DD/MM/AAAA HH:MM:SS)", "codigo_verificacao": "string",
              "prestador": {"cnpj": "string", "razao_social": "string", "inscricao_municipal": "string", "endereco": {"logradouro": "string", "bairro": "string", "cep": "string", "cidade": "string", "uf": "string"}},
              "tomador": {"cpf": "string", "razao_social": "string", "email": "string", "endereco": {"logradouro": "string", "bairro": "string", "cep": "string", "cidade": "string", "uf": "string"}},
              "servico": {"discriminacao": "string", "codigo": "string", "descricao": "string"},
              "valores": {"total_servico": "string (ex: R$ 1.200,00)", "base_calculo": "string (ex: 1.200,00)", "aliquota": "string (ex: 2,00%)", "valor_iss": "string (ex: 24,00)"}
            }
            """

            response = model.generate_content([prompt, content_to_process])
            json_text = response.text.replace('```json', '').replace('```', '').strip()
            dados_brutos = json.loads(json_text)

            dados_achatados = {
                "numero_nota": dados_brutos.get("numero_nota"), "data_hora_emissao": dados_brutos.get("data_hora_emissao"), "codigo_verificacao": dados_brutos.get("codigo_verificacao"),
                "prestador_cnpj": dados_brutos.get("prestador", {}).get("cnpj"), "prestador_razao_social": dados_brutos.get("prestador", {}).get("razao_social"), "prestador_inscricao_municipal": dados_brutos.get("prestador", {}).get("inscricao_municipal"),
                "prestador_logradouro": dados_brutos.get("prestador", {}).get("endereco", {}).get("logradouro"), "prestador_bairro": dados_brutos.get("prestador", {}).get("endereco", {}).get("bairro"), "prestador_cep": dados_brutos.get("prestador", {}).get("endereco", {}).get("cep"), "prestador_cidade": dados_brutos.get("prestador", {}).get("endereco", {}).get("cidade"), "prestador_uf": dados_brutos.get("prestador", {}).get("endereco", {}).get("uf"),
                "tomador_cpf": dados_brutos.get("tomador", {}).get("cpf"), "tomador_razao_social": dados_brutos.get("tomador", {}).get("razao_social"), "tomador_email": dados_brutos.get("tomador", {}).get("email"),
                "tomador_logradouro": dados_brutos.get("tomador", {}).get("endereco", {}).get("logradouro"), "tomador_bairro": dados_brutos.get("tomador", {}).get("endereco", {}).get("bairro"), "tomador_cep": dados_brutos.get("tomador", {}).get("endereco", {}).get("cep"), "tomador_cidade": dados_brutos.get("tomador", {}).get("endereco", {}).get("cidade"), "tomador_uf": dados_brutos.get("tomador", {}).get("endereco", {}).get("uf"),
                "discriminacao_servicos": dados_brutos.get("servico", {}).get("discriminacao"), "servico_codigo": dados_brutos.get("servico", {}).get("codigo"), "servico_descricao": dados_brutos.get("servico", {}).get("descricao"),
                "valor_total_servico": dados_brutos.get("valores", {}).get("total_servico"), "base_calculo": dados_brutos.get("valores", {}).get("base_calculo"), "aliquota": dados_brutos.get("valores", {}).get("aliquota"), "valor_iss": dados_brutos.get("valores", {}).get("valor_iss"),
            }

            dados_finais = clean_and_format_data(dados_achatados)
            dados_finais['hash'] = current_hash
            dados_finais['arquivo'] = filename
            dados_finais['data_processamento'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

            new_row = [dados_finais.get(header, '') for header in HEADERS]
            worksheet.append_row(new_row)

            print(f"✅ Dados de '{filename}' adicionados à planilha com sucesso!")
            time.sleep(2)

    print("\nProcesso concluído.")

except gspread.exceptions.SpreadsheetNotFound:
    print(f"ERRO: Planilha não encontrada.")
except Exception as e:
    print(f"Ocorreu um erro inesperado: {e}")